Awodey 
Riehl

Direct axiomaiztzation of functors


# GAT

In [1]:
from dataclasses import dataclass
from kdrag.all import *
@dataclass
class GATExpr():
    vs: list[smt.ExprRef]
    ctx: list[smt.BoolRef]
    t: smt.ExprRef
    @classmethod
    def Const(name, sort, vs=[], ctx=[]):
        return GATExpr(vs, ctx, smt.Const(name, sort))
    def merge_ctx(self, y: GATExpr):
        self.vs.extend(y.vs)
        self.ctx.extend(y.ctx)
    def __get_attr__(self, k):
        attr = getattr(self.t, k)
        if not callable(attr):
            return GATExpr(self.vs, self.ctx, attr)
        else:
            def wrapper(*args, **kwargs):
                res = GATExpr(attr(*args, **kwargs))
                for x in args:
                    if isinstance(x,GATExpr):
                        res.merge_ctx(x)
                for v in kwargs.values():
                    if isinstance(v,GATExpr):
                        res.merge_ctx(v)
                return res
            return wrapper

def GATConst(name, sort, vs=[], ctx=[]):
    return GATExpr.Const(name,sort,vs=vs,ctx=ctx)




ModuleNotFoundError: No module named 'kdrag'

# Basic Cat

In [1]:
from kdrag.all import *
Ob = smt.DeclareSort("Ob")
Arr = smt.DeclareSort("Arr")
comp = smt.Function("comp", Arr,Arr,Arr)
id = smt.Function("id", Ob,Arr)
dom = smt.Function("dom", Arr,Ob)
cod = smt.Function("cod", Arr,Ob)
kd.notation.matmul.register(Arr, comp)
a,b,c,d = smt.Consts("a b c d", Ob)
f,g,h,k = smt.Consts("f g h k", Arr)
wf = smt.Function("wf", Arr, smt.BoolSort())
kd.notation.wf.register(Arr, wf)

catdb = []

wf_comp = kd.axiom(kd.QForAll([f,g], cod(g) == dom(f), wf(f @ g)))
wf_id = kd.axiom(smt.ForAll([a], wf(id(a))))


id_left = kd.axiom(kd.QForAll([f], f @ id(dom(f)) == f))
id_right = kd.axiom(kd.QForAll([f], id(cod(f)) @ f == f))
comp_assoc = kd.axiom(kd.QForAll([f,g,h], dom(f) == cod(g), dom(g) == cod(h) , f @ (g @ h) == (f @ g) @ h))

dom_comp = kd.axiom(kd.QForAll([f,g], wf(f @ g) , dom(f @ g) == dom(g)))
cod_comp = kd.axiom(kd.QForAll([f,g], wf(f @ g) , cod(f @ g) == cod(f)))

catdb.extend([wf_comp,wf_id,id_left,id_right,comp_assoc, dom_comp, cod_comp])


Admitting lemma ForAll([x, y, z], mul(x, mul(y, z)) == mul(mul(x, y), z))
Admitting lemma ForAll(x, Implies(x >= 0, sqrt(x) >= 0))
Admitting lemma ForAll(x, Implies(x >= 0, sqrt(x)**2 == x))
Admitting lemma ForAll(x, Implies(x >= 0, sqrt(sqr(x)) == x))


I think I want a theorem constructor
CForAll(  ) which adds wf over all subterms?

In [26]:
catdb

[|- ForAll([f, g],
        Implies(And(wf(f), wf(g), cod(g) == dom(f)),
                wf(comp(f, g)))),
 |- ForAll(a, wf(id(a))),
 |- ForAll(f, Implies(wf(f), comp(f, id(dom(f))) == f)),
 |- ForAll(f, Implies(wf(f), comp(id(cod(f)), f) == f)),
 |- ForAll([f, g, h],
        Implies(And(wf(f),
                    wf(g),
                    wf(h),
                    dom(f) == cod(g),
                    dom(g) == cod(h)),
                comp(f, comp(g, h)) == comp(comp(f, g), h))),
 |- ForAll([f, g],
        Implies(And(wf(f), wf(g), wf(comp(f, g))),
                dom(comp(f, g)) == dom(g))),
 |- ForAll([f, g],
        Implies(And(wf(f), wf(g), wf(comp(f, g))),
                cod(comp(f, g)) == cod(f)))]

In [2]:
def clemma(thm, by=[]):
    #rest all well formed as lemma
    return kd.kernel.lemma(thm, by=by + catdb)

clemma(smt.BoolVal(False)) # ok so we don't have a contradiction 
 

LemmaError: (False, 'Countermodel', [wf = [else -> True],
 cod = [else -> Ob!val!0],
 comp = [else ->
         If(And(wf(Var(1)), Var(0) == id(cod(Var(1)))),
            Var(1),
            Var(0))],
 id = [else -> Arr!val!0],
 dom = [else -> Ob!val!0]])

In [3]:
clemma(kd.QForAll([f,g,h,k], dom(f) == cod(g), dom(g) == cod(h), dom(h) == cod(k), f @ (g @ (h @ k)) == (f @ g) @ (h @ k)))


|- ForAll([f, g, h, k],
       Implies(And(wf(f),
                   wf(g),
                   wf(h),
                   wf(k),
                   dom(f) == cod(g),
                   dom(g) == cod(h),
                   dom(h) == cod(k)),
               comp(f, comp(g, comp(h, k))) ==
               comp(comp(f, g), comp(h, k))))

In [5]:
monic = kd.define("monic", [f], kd.QForAll([g,h], dom(f) == cod(g), dom(f) == cod(h), 
                                            f @ g == f @ h, g == h))
epic = kd.define("epic", [f], kd.QForAll([g,h], dom(g) == cod(f), dom(h) == cod(f),
                                            g @ f == h @ f, g == h))

catdb.extend([monic.defn, epic.defn])

In [ ]:
iso_arr = kd.define("iso_arr", [f,g], )
iso_ob = kd.define("iso_ob", [a,b], kd.QExists([f,g], dom(f) == a, cod(f) == b,
                                                      dom(g) == b, cod(g) == a,
                                                      f @ g == id(b), g @ f == id(a)))

In [7]:
# https://en.wikipedia.org/wiki/Initial_and_terminal_objects
initial = kd.define("initial", [a], kd.QForAll([b], kd.QExists([f], cod(f) == b, dom(f) == a, 
                                                                 kd.QForAll([g], cod(g) == b, dom(g) == a,  
                                                                              g == f))))
terminal = kd.define("terminal", [a], kd.QForAll([b], kd.QExists([f], cod(f) == a, dom(f) == b,
                                                                    kd.QForAll([g], cod(g) == a, dom(g) == b,
                                                                                g == f))))

# two terminal objects are isomorphic

In [ ]:
proj1 = smt.("proj1", Ob, Arr)
proj2 = smt.Function("proj2", Ob, Arr)
is_prod = kd.define("is_prod", [a,b,c], smt.ForAll([d], smt.Exists([f,g], smt.And(comp(f, a, d), comp(g, b, d), smt.ForAll([h], comp(h, c, h) == smt.And(comp(h, f, h), comp(h, g, h))))))

par = smt.Function("par", Arr, Arr, Arr)



terminal 
initial
pullback = 

cartesian = kd.define("cartesian", [], kd.QForAll([a,b], kd.QExists([c], kd.Prod(a,b,c))))
closed = 

axiom about particular arrangement can be written as
QForAll([], cartesian, closed, )


# CAT - 2category

Could use endofunctor? Endofunctor + domain of objects? + cod

Diurectly axiomiatiZe 2 category. How much functor nonsense does that give us.




In [ ]:
Arr2 = smt.DeclareSort("Arr2")
comp2 = smt.Function("comp2", Arr2, Arr2, Arr2)
# could use notation overload
id2 = smt.Function("id2", Arr, Arr2)
dom2 = smt.Function("dom2", Arr2, Arr)
cod2 = smt.Function("cod2", Arr2, Arr)
